# Thesis 2020-2021: DeepMoji Model 

In this notebook, we will create a deepmoji model.

In [26]:
conda install pytorch -c pytorch

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install -e .

Obtaining file:///C:/Users/Admin/Desktop/Master%20thesis/Thesis_2021/deepmoji
  Using cached numpy-1.13.1.zip (5.0 MB)
  Using cached scipy-0.19.1.tar.gz (14.1 MB)
  Using cached scikit-learn-0.19.0.tar.gz (9.3 MB)
  Using cached text_unidecode-1.0-py2.py3-none-any.whl (75 kB)
  Running setup.py clean for numpy
  Running setup.py clean for scipy
  Running setup.py clean for scikit-learn
Failed to build numpy scipy scikit-learn
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
    Running setup.py install for numpy: started
    Running setup.py install for numpy: finished with status 'error'
  Rolling back uninstall of numpy
  Moving to c:\users\admin\anaconda3\lib\site-packages\numpy-1.19.2.dist-info\
   from C:\Users\Admin\anaconda3\Lib\site-packages\~umpy-1.19.2.dist-info
  Moving to c:\users\admin\anaconda3\lib\site-packages\numpy\
   from C:\Users\Admin\anaconda3\Lib\site-packages\~

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Admin\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Admin\\AppData\\Local\\Temp\\pip-install-ijxiodyu\\numpy\\setup.py'"'"'; __file__='"'"'C:\\Users\\Admin\\AppData\\Local\\Temp\\pip-install-ijxiodyu\\numpy\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Admin\AppData\Local\Temp\pip-wheel-ua9kqtjr'
       cwd: C:\Users\Admin\AppData\Local\Temp\pip-install-ijxiodyu\numpy\
  Complete output (157 lines):
  Running from numpy source directory.
  C:\Users\Admin\AppData\Local\Temp\pip-install-ijxiodyu\numpy\numpy\distutils\misc_util.py:469: SyntaxWarning: "is" with a literal. Did you mean "=="?
    return is_string(s) and ('*' in s or '?' is s)
  blas_opt_info:
  blas_mkl_info:
    libraries mkl_rt not found in ['C:\\Users\\A

    NOT AVAILABLE
  
  openblas_info:
    libraries openblas not found in ['C:\\Users\\Admin\\anaconda3\\lib', 'C:\\', 'C:\\Users\\Admin\\anaconda3\\libs']
  get_default_fcompiler: matching types: '['gnu', 'intelv', 'absoft', 'compaqv', 'intelev', 'gnu95', 'g95', 'intelvem', 'intelem', 'flang']'
  customize GnuFCompiler
  Could not locate executable g77
  Could not locate executable f77
  customize IntelVisualFCompiler
  Could not locate executable ifort
  Could not locate executable ifl
  customize AbsoftFCompiler
  Could not locate executable f90
  customize CompaqVisualFCompiler
  Could not locate executable DF
  customize IntelItaniumVisualFCompiler
  Could not locate executable efl
  customize Gnu95FCompiler
  Could not locate executable gfortran
  Could not locate executable f95
  customize G95FCompiler
  Could not locate executable g95
  customize IntelEM64VisualFCompiler
  customize IntelEM64TFCompiler
  Could not locate executable efort
  Could not locate executable efc
  cust

In [28]:
""" Use torchMoji to encode texts into emotional feature vectors.
"""
from __future__ import print_function, division, unicode_literals
import json
from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_feature_encoding # THIS LINE KILLS MY KERNEL
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH


TEST_SENTENCES = ['I love mom\'s cooking',
                  'I love how you never reply back..',
                  'I love cruising with my homies',
                  'I love messing with yo mind!!',
                  'I love you and now you\'re just gone..',
                  'This is shit',
                  'This is the shit']

maxlen = 30
batch_size = 32

print('Tokenizing using dictionary from {}'.format(VOCAB_PATH))
with open(VOCAB_PATH, 'r') as f:
    vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, maxlen)
tokenized, _, _ = st.tokenize_sentences(TEST_SENTENCES)
print("TOKENIZED:",tokenized)
print('Loading model from {}.'.format(PRETRAINED_PATH))
model = torchmoji_feature_encoding(PRETRAINED_PATH)
print(model)

print('Encoding texts..')
encoding = model(tokenized)

print('First 5 dimensions for sentence: {}'.format(TEST_SENTENCES[0]))
print(encoding[0,:5])

# Now you could visualize the encodings to see differences,
# run a logistic regression classifier on top,
# or basically anything you'd like to do.

Tokenizing using dictionary from C:\Users\Admin\Desktop\Master thesis\Thesis_2021\deepmoji/model/vocabulary.json



  copying sklearn\manifold\tests\test_locally_linear.py -> build\lib.win-amd64-3.8\sklearn\manifold\tests
  copying sklearn\manifold\tests\test_mds.py -> build\lib.win-amd64-3.8\sklearn\manifold\tests
  copying sklearn\manifold\tests\test_spectral_embedding.py -> build\lib.win-amd64-3.8\sklearn\manifold\tests
  copying sklearn\manifold\tests\test_t_sne.py -> build\lib.win-amd64-3.8\sklearn\manifold\tests
  copying sklearn\manifold\tests\__init__.py -> build\lib.win-amd64-3.8\sklearn\manifold\tests
  creating build\lib.win-amd64-3.8\sklearn\metrics
  copying sklearn\metrics\base.py -> build\lib.win-amd64-3.8\sklearn\metrics
  copying sklearn\metrics\classification.py -> build\lib.win-amd64-3.8\sklearn\metrics
  copying sklearn\metrics\pairwise.py -> build\lib.win-amd64-3.8\sklearn\metrics
  copying sklearn\metrics\ranking.py -> build\lib.win-amd64-3.8\sklearn\metrics
  copying sklearn\metrics\regression.py -> build\lib.win-amd64-3.8\sklearn\metrics
  copying sklearn\metrics\scorer.py -

TOKENIZED: [[   18    87  3497  1864     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   18    87    79    13   122   935    85    34     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   18    87 11824    31    41  5113     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   18    87  3745    31  2324   440    50     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   18    87    13    12   111   102    42   642    34     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0]
 [   22    28   172     0     0     0     0     0     

    Setting PTATLAS=ATLAS
      libraries ptf77blas,ptcblas,atlas not found in ['C:\\Users\\Admin\\anaconda3\\lib', 'C:\\', 'C:\\Users\\Admin\\anaconda3\\libs']
      NOT AVAILABLE
    
    atlas_blas_info:
      libraries f77blas,cblas,atlas not found in ['C:\\Users\\Admin\\anaconda3\\lib', 'C:\\', 'C:\\Users\\Admin\\anaconda3\\libs']
      NOT AVAILABLE
    
    C:\Users\Admin\AppData\Local\Temp\pip-install-ijxiodyu\numpy\numpy\distutils\system_info.py:572: UserWarning:
        Atlas (http://math-atlas.sourceforge.net/) libraries not found.
        Directories to search for the libraries can be specified in the
        numpy/distutils/site.cfg file (section [atlas]) or by setting
        the ATLAS environment variable.
      self.calc_info()
    blas_info:
      libraries blas not found in ['C:\\Users\\Admin\\anaconda3\\lib', 'C:\\', 'C:\\Users\\Admin\\anaconda3\\libs']
      NOT AVAILABLE
    
    C:\Users\Admin\AppData\Local\Temp\pip-install-ijxiodyu\numpy\numpy\distutils\system_in

Loading weights for embed.weight
Loading weights for lstm_0.weight_ih_l0
Loading weights for lstm_0.weight_hh_l0
Loading weights for lstm_0.bias_ih_l0
Loading weights for lstm_0.bias_hh_l0
Loading weights for lstm_0.weight_ih_l0_reverse
Loading weights for lstm_0.weight_hh_l0_reverse
Loading weights for lstm_0.bias_ih_l0_reverse
Loading weights for lstm_0.bias_hh_l0_reverse
Loading weights for lstm_1.weight_ih_l0
Loading weights for lstm_1.weight_hh_l0
Loading weights for lstm_1.bias_ih_l0
Loading weights for lstm_1.bias_hh_l0
Loading weights for lstm_1.weight_ih_l0_reverse
Loading weights for lstm_1.weight_hh_l0_reverse
Loading weights for lstm_1.bias_ih_l0_reverse
Loading weights for lstm_1.bias_hh_l0_reverse
Loading weights for attention_layer.attention_vector
Ignoring weights for output_layer.0.weight
Ignoring weights for output_layer.0.bias
TorchMoji(
  (embed): Embedding(50000, 256)
  (embed_dropout): Dropout2d(p=0, inplace=False)
  (lstm_0): LSTMHardSigmoid(256, 512, batch_first

ValueError: too many values to unpack (expected 2)